# Pyvista examples

In [1]:
from matplotlib.colors import ListedColormap
import matplotlib.pyplot as plt
import numpy as np

import pyvista as pv
from pyvista import examples

## Playing Around with Colormaps

In [2]:
mesh = examples.download_st_helens().warp_by_scalar()
# Add scalar array with range (0, 100) that correlates with elevation
#mesh['values'] = pv.plotting.normalize(mesh['Elevation']) * 100
mesh['values'] = pv.plotting.normalize(mesh['Elevation']) * 100

In [3]:
# Define the colors we want to use
blue = np.array([12 / 256, 238 / 256, 246 / 256, 1.0])
black = np.array([11 / 256, 11 / 256, 11 / 256, 1.0])
grey = np.array([189 / 256, 189 / 256, 189 / 256, 1.0])
yellow = np.array([255 / 256, 247 / 256, 0 / 256, 1.0])
red = np.array([1.0, 0.0, 0.0, 1.0])

mapping = np.linspace(mesh['values'].min(), mesh['values'].max(), 256)
newcolors = np.empty((256, 4))
newcolors[mapping >= 80] = red
newcolors[mapping < 80] = grey
newcolors[mapping < 55] = yellow
newcolors[mapping < 30] = blue
newcolors[mapping < 1] = black

# Make the colormap from the listed colors
my_colormap = ListedColormap(newcolors)

In [4]:
mesh.plot(scalars='values', cmap=my_colormap)

Widget(value="<iframe src='http://localhost:62731/index.html?ui=P_0x15625cc10_0&reconnect=auto' style='width: …

## Read OpenFOAM Example

In [5]:
filename = examples.download_cavity(load=False)
reader = pv.POpenFOAMReader(filename)

In [6]:
print(f"All patch names: {reader.patch_array_names}")
print(f"All patch status: {reader.all_patch_arrays_status}")

All patch names: ['internalMesh', 'patch/movingWall', 'patch/fixedWalls', 'patch/frontAndBack']
All patch status: {'internalMesh': True, 'patch/movingWall': True, 'patch/fixedWalls': True, 'patch/frontAndBack': True}


In [7]:
mesh = reader.read()
print(f"Mesh patches: {mesh.keys()}")
internal_mesh = mesh["internalMesh"]  # or internal_mesh = mesh[0]

Mesh patches: ['internalMesh', 'boundary']


In [8]:
print(internal_mesh)

UnstructuredGrid (0x156d170a0)
  N Cells:    400
  N Points:   882
  X Bounds:   0.000e+00, 1.000e-01
  Y Bounds:   0.000e+00, 1.000e-01
  Z Bounds:   0.000e+00, 1.000e-02
  N Arrays:   4



In [9]:
boundaries = mesh["boundary"]
print(boundaries)
print(f"Boundaries patches: {boundaries.keys()}")
print(boundaries["movingWall"])

MultiBlock (0x156d17940)
  N Blocks:	3
  X Bounds:	0.000, 0.100
  Y Bounds:	0.000, 0.100
  Z Bounds:	0.000, 0.010

Boundaries patches: ['movingWall', 'fixedWalls', 'frontAndBack']
PolyData (0x156d17c40)
  N Cells:    20
  N Points:   42
  N Strips:   0
  X Bounds:   0.000e+00, 1.000e-01
  Y Bounds:   1.000e-01, 1.000e-01
  Z Bounds:   0.000e+00, 1.000e-02
  N Arrays:   4



In [10]:
print("Cell Data:")
print(internal_mesh.cell_data)
print("\nPoint Data:")
print(internal_mesh.point_data)

Cell Data:
pyvista DataSetAttributes
Association     : CELL
Active Scalars  : p
Active Vectors  : U
Active Texture  : None
Active Normals  : None
Contains arrays :
    U                       float32    (400, 3)             VECTORS
    p                       float32    (400,)               SCALARS

Point Data:
pyvista DataSetAttributes
Association     : POINT
Active Scalars  : p
Active Vectors  : U
Active Texture  : None
Active Normals  : None
Contains arrays :
    U                       float32    (882, 3)             VECTORS
    p                       float32    (882,)               SCALARS


In [11]:
reader.cell_to_point_creation = False
internal_mesh = reader.read()["internalMesh"]
print("Cell Data:")
print(internal_mesh.cell_data)
print("\nPoint Data:")
print(internal_mesh.point_data)

Cell Data:
pyvista DataSetAttributes
Association     : CELL
Active Scalars  : p
Active Vectors  : U
Active Texture  : None
Active Normals  : None
Contains arrays :
    U                       float32    (400, 3)             VECTORS
    p                       float32    (400,)               SCALARS

Point Data:
pyvista DataSetAttributes
Association     : POINT
Active Scalars  : None
Active Vectors  : None
Active Texture  : None
Active Normals  : None
Contains arrays : None


In [12]:
print(f"Available Time Values: {reader.time_values}")
reader.set_active_time_value(2.5)
reader.cell_to_point_creation = True  # Need point data for streamlines
mesh = reader.read()
internal_mesh = mesh["internalMesh"]
boundaries = mesh["boundary"]

Available Time Values: [0.0, 0.5, 1.0, 1.5, 2.0, 2.5]


In [13]:
def slice_z_center(mesh):
    """Slice mesh through center in z normal direction, move to z=0."""
    slice_mesh = mesh.slice(normal='z')
    slice_mesh.translate((0, 0, -slice_mesh.center[-1]), inplace=True)
    return slice_mesh


slice_internal_mesh = slice_z_center(internal_mesh)
slice_boundaries = pv.MultiBlock(
    {key: slice_z_center(boundaries[key]) for key in boundaries.keys()}
)

In [14]:
streamlines = slice_internal_mesh.streamlines_evenly_spaced_2D(
    vectors='U',
    start_position=(0.05, 0.05, 0),
    separating_distance=1,
    separating_distance_ratio=0.1,
)

In [15]:
plotter = pv.Plotter()
plotter.add_mesh(slice_boundaries["movingWall"], color='red', line_width=3)
plotter.add_mesh(slice_boundaries["fixedWalls"], color='black', line_width=3)
plotter.add_mesh(streamlines.tube(radius=0.0005), scalars="U")
plotter.view_xy()
plotter.enable_parallel_projection()
plotter.show()

Widget(value="<iframe src='http://localhost:62731/index.html?ui=P_0x156e7be20_1&reconnect=auto' style='width: …

### No streamlines

In [16]:
pt = pv.Plotter()
pt.add_mesh(slice_internal_mesh)
pt.show()

Widget(value="<iframe src='http://localhost:62731/index.html?ui=P_0x156e77e50_2&reconnect=auto' style='width: …

In [17]:
pt = pv.Plotter()
pt.add_mesh(internal_mesh)
pt.show()

Widget(value="<iframe src='http://localhost:62731/index.html?ui=P_0x156e7b940_3&reconnect=auto' style='width: …